In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # plot figures
import seaborn as sns # plot figures
import plotly.express as px
from datetime import datetime
from collections import Counter
from matplotlib.ticker import MaxNLocator
import matplotlib.gridspec as gridspec

import math
import random
import os
import time

import warnings
warnings.filterwarnings('ignore') 

In [ ]:
raw_patient = pd.read_csv("cats_data_extract_20210920_master_deid.csv")
raw_patient = raw_patient.reset_index(drop = True)

In [ ]:
raw_patient.head(2)

In [ ]:
len(raw_patient)

 ## Select patients transported after First July, 2016

In [ ]:
raw_patient = raw_patient[34:-1]

# Remove blank in the column name

In [ ]:
raw_patient.rename(str.strip, axis='columns',inplace=True)

In [ ]:
raw_patient.columns

## Filter patients who's referral is accepted for CATS transport

In [ ]:
df_all_patients = raw_patient[raw_patient.referral_outcome.str.contains('Accepted',na=False)]

In [ ]:
df_all_patients = df_all_patients.reset_index(drop = True)

In [ ]:
print("length of df_all_patients:",len(df_all_patients))

In [ ]:
# Calcualte how many patients have vital sign data files

In [ ]:
print(len(df_all_patients[df_all_patients["csv_files"] == df_all_patients["csv_files"]]))

In [ ]:
df_all_patients.head(3)

In [ ]:
df_all_patients.isnull().sum()

# Add a destination Column
- if the transport dest is great ormand hospital london (GOSH)

In [ ]:
hospital_name_list = df_all_patients['Destination Unit'].unique().tolist()
hospital_name_list

In [ ]:
# Check the nan value's data type - nan-> float
nanElement = [x for x in hospital_name_list if isinstance(x,float)]
nanElement[0]

In [ ]:
# Remove nan values in hospital name
hospital_name_list.remove(nanElement[0])
hospital_name_list

In [ ]:
matching = [s for s in hospital_name_list if "great ormond" in s.lower()]
matching

In [ ]:
for i in range(0,len(df_all_patients)):
    if df_all_patients.iloc[i]['Destination Unit'] in matching:
        df_all_patients.at[i,'dest_is_GOSH'] = 'Yes'
        #print(df_all_patients.iloc[i]['Destination Unit'])
    else:
        df_all_patients.at[i,'dest_is_GOSH'] = 'No'

In [ ]:
df_all_patients['dest_is_GOSH'].unique()

In [ ]:
print('The number of transports to GOSH is: ',len(df_all_patients[df_all_patients['dest_is_GOSH'] == 'Yes']))

In [ ]:
df_all_patients.head(3)

In [ ]:
len(df_all_patients)

## We can notice that there are missing values and blank/null values in some columns.
- Applying appropriate methods to fill in missing values

#### Check gender

In [ ]:
df_all_patients['gender'].unique()

In [ ]:
gender_missing = df_all_patients[df_all_patients['gender'] != df_all_patients['gender']]
gender_missing

In [ ]:
df_all_patients = df_all_patients.drop(gender_missing.index)

# Delete gender = 'Ambiguous', only 1 patient
df_all_patients = df_all_patients[df_all_patients['gender'] != 'Ambiguous']

# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

#### Check age_mon

In [ ]:
age_missing = df_all_patients[df_all_patients['age_mon'] != df_all_patients['age_mon']]
len(age_missing)

In [ ]:
age_missing.index

In [ ]:
df_all_patients = df_all_patients.drop(age_missing.index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

In [ ]:
# We remove 2 records, so the corrent length should be 6471-2 = 6469
print('after removing nan age_mon: ', len(df_all_patients))

#### Check PIM3 Score

In [ ]:
PIM3_missing = df_all_patients[df_all_patients['pim3'] == 'Error in PIM3 Score']
PIM3_missing[['cats_id','pim3']]

In [ ]:
df_all_patients = df_all_patients.drop(PIM3_missing.index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

In [ ]:
# We remove 2 records, so the corrent length should be 6471-2 = 6469
print('after removing abnormal PIM3 score: ', len(df_all_patients))

In [ ]:
df_all_patients

#### Check vaso_agents
- In this column, only the patient who has VASO will be labled as 'Yes'. Other cases would be 'None', including 'None' and NaN

In [ ]:
df_all_patients[['vasoactive_agent_used','vasoactive_agent_name']]

In [ ]:
for i in range(0,len(df_all_patients)):
    # If value is Yes, the patient has vaso drug; otherwise, the patient has no vaso drugs
    if df_all_patients.iloc[i]['vasoactive_agent_used'] != 'Yes':
        df_all_patients.at[i,'vasoactive_agent_used'] = 'None'

In [ ]:
df_all_patients[['vasoactive_agent_used','vasoactive_agent_name']].isnull().sum()

In [ ]:
df_all_patients['vasoactive_agent_used'].unique()

In [ ]:
# We remove 2 records, so the corrent length should be 6471-2 = 6469
print('after cleaning VASO: ', len(df_all_patients))

#### Check local_interv

In [ ]:
df_all_patients['local_interv'].isnull().sum()

In [ ]:
for i in range(0,len(df_all_patients)):
    # If value is Yes, the patient has vaso drug; otherwise, the patient has no vaso drugs
    if df_all_patients.iloc[i]['local_interv'] != df_all_patients.iloc[i]['local_interv']:
        df_all_patients.at[i,'local_interv'] = 'None'

In [ ]:
df_all_patients['local_interv'].isnull().sum()

In [ ]:
# We remove 2 records, so the corrent length should be 6471-2 = 6469
print('after cleaning local_interv: ', len(df_all_patients))

#### Check cats_interv

In [ ]:
df_all_patients['cats_interv'].isnull().sum()

In [ ]:
for i in range(0,len(df_all_patients)):
    # If value is Yes, the patient has vaso drug; otherwise, the patient has no vaso drugs
    if df_all_patients.iloc[i]['cats_interv'] != df_all_patients.iloc[i]['cats_interv']:
        df_all_patients.at[i,'cats_interv'] = 'None'

In [ ]:
df_all_patients['cats_interv'].isnull().sum()

In [ ]:
df_all_patients['cats_interv'].unique()

In [ ]:
# We remove 2 records, so the corrent length should be 6471-2 = 6469
print('after cleaning cats_interv: ', len(df_all_patients))

#### Check inh_no

In [ ]:
df_all_patients['inhaled_no'].isnull().sum()

In [ ]:
for i in range(0,len(df_all_patients)):
    # If value is Yes, the patient has vaso drug; otherwise, the patient has no vaso drugs
    if df_all_patients.iloc[i]['inhaled_no'] != df_all_patients.iloc[i]['inhaled_no']:
        df_all_patients.at[i,'inhaled_no'] = 'No'

In [ ]:
df_all_patients['inhaled_no'].isnull().sum()

In [ ]:
df_all_patients['inhaled_no'].unique()

In [ ]:
print('after cleaning cats_interv: ', len(df_all_patients))

#### Check vent_status

In [ ]:
df_all_patients['ventilation_status'].isnull().sum()

In [ ]:
for i in range(0,len(df_all_patients)):
    # If value is Yes, the patient has vaso drug; otherwise, the patient has no vaso drugs
    if df_all_patients.iloc[i]['ventilation_status'] != df_all_patients.iloc[i]['ventilation_status']:
        df_all_patients.at[i,'ventilation_status'] = 'No'

In [ ]:
df_all_patients['ventilation_status'].isnull().sum()

In [ ]:
df_all_patients['ventilation_status'].unique()

In [ ]:
# We remove 2 records, so the corrent length should be 6471-2 = 6469
print('after cleaning ventilation_status: ', len(df_all_patients))

### Critical Incident Type Label

In [ ]:
df_all_patients["incident_type"] = df_all_patients["incident_type"].apply(lambda x: 'None' if x!=x else x)

In [ ]:
df_all_patients["incident_type"]

In [ ]:
df_all_patients["incident_type"].isnull().sum()

In [ ]:
df_all_patients['incident_type'].unique()

## Add pateint/equipment related CI number to dataframe
- if value is 0, it means no CI happened during transport for the patient
- if value is non-zero number n, it means CI happened N times during transport.

In [ ]:
df_all_patients['CI_Patient_Equipment_Related'] = df_all_patients['incident_type'].apply(lambda x: 1 if 'Patient' in x or 'Equipment' in x else 0)

In [ ]:
df_all_patients['CI_Patient_Related'] = df_all_patients['incident_type'].apply(lambda x: Counter(x.split(','))['Patient related'] if 'Patient' in x else 0)

In [ ]:
df_all_patients['CI_Ambulance_Related'] = df_all_patients['incident_type'].apply(lambda x: Counter(x.split(','))['Ambulance related'] if 'Ambulance' in x else 0)

In [ ]:
df_all_patients['CI_Equipment_Related'] = df_all_patients['incident_type'].apply(lambda x: Counter(x.split(','))['Equipment related'] if 'Equipment' in x else 0)

In [ ]:
df_all_patients['CI_Communication_Related'] = df_all_patients['incident_type'].apply(lambda x: Counter(x.split(','))['Communication/Delays'] if 'Communication' in x else 0)

In [ ]:
CI_patients = df_all_patients[['cats_id','CI_Patient_Equipment_Related','CI_Patient_Related','CI_Ambulance_Related','CI_Equipment_Related','CI_Communication_Related']]

In [ ]:
# Pateints who have critical events over 3 times
CI_patients[CI_patients['CI_Patient_Related'] == 3]

### Check null data in df_all_patients

In [ ]:
df_all_patients.isnull().sum()

#### Check pim3 null data

In [ ]:
pim_null_data = df_all_patients[df_all_patients['pim3'] != df_all_patients['pim3']]
pim_null_data

In [ ]:
df_all_patients = df_all_patients.drop(pim_null_data.index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

#### Timmestamp of Check Destination Unite

In [ ]:
depart_null_data = df_all_patients[df_all_patients['Depart Collection Unit'] != df_all_patients['Depart Collection Unit']]
depart_null_data

In [ ]:
df_all_patients = df_all_patients.drop(depart_null_data.index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

#### Check Timestamp of Arrive Destination Unit

In [ ]:
Arrive_null_data = df_all_patients[df_all_patients['Arrive Destination Unit'] != df_all_patients['Arrive Destination Unit']]
Arrive_null_data

In [ ]:
df_all_patients = df_all_patients.drop(Arrive_null_data.index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

In [ ]:
print('after cleaning df_monitor_patients ', len(df_all_patients))

### Check arrive_destunit null data

In [ ]:
df_all_patients = df_all_patients[df_all_patients['arrive_collunit'] == df_all_patients['arrive_collunit']]
df_all_patients = df_all_patients.reset_index(drop = True)

### Check depart_destunit null data

In [ ]:
depart_destunit_null_data = df_all_patients[df_all_patients['depart_destunit'] != df_all_patients['depart_destunit']]
depart_destunit_null_data

In [ ]:
df_all_patients = df_all_patients[df_all_patients['depart_destunit'] == df_all_patients['depart_destunit']]
df_all_patients = df_all_patients.reset_index(drop = True)

In [ ]:
print('after cleaning df_monitor_patients ', len(df_all_patients))

### Remove Patients aged larger than 18 years old

In [ ]:
# 18 years old :  18 * 12 =216 months
df_all_patients = df_all_patients[df_all_patients['age_mon'] <= 18*12]
df_all_patients = df_all_patients.reset_index(drop=True)
print(len(df_all_patients))

### Add "age_group" column in the df_all_patients

In [ ]:
for i in range(0,len(df_all_patients)):
    age = df_all_patients.iloc[i]['age_mon']
    if age>=0 and age<=1:
        df_all_patients.at[i,'age_group'] = 'Newborn'
    elif age>1 and age<=12:
        df_all_patients.at[i,'age_group'] = 'Infant'
    elif age>12 and age<=12*4:
        df_all_patients.at[i,'age_group'] = 'Pre-school'
    elif age>12*4 and age<=12*11:
        df_all_patients.at[i,'age_group'] = 'School'
    elif age>12*11 and age<=12*18:
        df_all_patients.at[i,'age_group'] = 'Adolescens'

In [ ]:
df_all_patients.age_group.unique()

### Create Stablization, Journey and Handover Columns

#### Correct two records
- cats_id = 34388, Arrive Destination Unit = '27/04/2018 17:00'
- cats_id = 36764, Arrive Destination Unit = '28/04/2018 23:30'

In [ ]:
patient_34388_index = df_all_patients[df_all_patients['cats_id'] == 34388].index[0] # deidentified ID
patient_34388_index

In [ ]:
df_all_patients.at[patient_34388_index,['Arrive Destination Unit']] = '27/04/2018 17:00'

In [ ]:
df_all_patients[df_all_patients['cats_id'] == 34388] # deidentified ID

In [ ]:
patient_36764_index = df_all_patients[df_all_patients['cats_id'] == 36764].index[0] # deidentified ID
patient_36764_index

In [ ]:
df_all_patients.at[patient_36764_index,['Arrive Destination Unit']] = '28/04/2018 23:30'

In [ ]:
df_all_patients[df_all_patients['cats_id'] == 36764] # deidentified ID

#### Delete one transport record
- because "Depart Collection Unit" is early than "arrive_collunit"

In [ ]:
patient_index = df_all_patients[df_all_patients['cats_id'] == 34051].index[0] # deidentified ID
patient_index

In [ ]:
df_all_patients = df_all_patients.drop(patient_index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

In [ ]:
df_all_patients['stablization'] = df_all_patients.apply(lambda x: (datetime.strptime(x['Depart Collection Unit'],'%d/%m/%Y %H:%M') - datetime.strptime(x['arrive_collunit'],'%d/%m/%Y %H:%M')).seconds//60,axis = 1)

In [ ]:
df_all_patients['journey'] = df_all_patients.apply(lambda x: (datetime.strptime(x['Arrive Destination Unit'],'%d/%m/%Y %H:%M') - datetime.strptime(x['Depart Collection Unit'],'%d/%m/%Y %H:%M')).seconds//60,axis = 1)

In [ ]:
df_all_patients['handover'] = df_all_patients.apply(lambda x: (datetime.strptime(x['depart_destunit'],'%d/%m/%Y %H:%M') - datetime.strptime(x['Arrive Destination Unit'],'%d/%m/%Y %H:%M')).seconds//60,axis = 1)

### Check Stablization outlier values

In [ ]:
stab_outlier_transport = df_all_patients[df_all_patients['stablization'] >= 400]
stab_outlier_transport

In [ ]:
journey_outlier_transport = df_all_patients[df_all_patients['journey'] >= 400]
journey_outlier_transport.head(3)

In [ ]:
handover_outlier_transport = df_all_patients[df_all_patients['handover'] >= 400]
handover_outlier_transport.head(3)

### Check df_monitor_patients Stablization outlier values

In [ ]:
stab_outlier_transport = df_all_patients[df_all_patients['stablization'] >= 400]
stab_outlier_transport

In [ ]:
patient_index = df_all_patients[df_all_patients['cats_id'] == 38156].index[0] # deidentified ID
patient_index

In [ ]:
# This patietn has a hearly 10 hours stablization time.
# This value is far larger than the average value, so we treat it as an outlier
# Remove this value
df_all_patients = df_all_patients.drop(patient_index)
# We need reset_index everytime after we delete or add records in the dataframe for correct iteration
df_all_patients = df_all_patients.reset_index(drop = True)

## Extract Patients having Vital Sign data

In [ ]:
df_file_check = pd.read_csv("Numerical_Vital_Sign_Files_Summary.csv")
df_monitor_patients = df_all_patients.merge(df_file_check, left_on='cats_id', right_on='cats_id')
df_monitor_patients = df_monitor_patients.reset_index(drop=True)

In [ ]:
print(len(df_all_patients))
print(len(df_monitor_patients))

## Calculate total transport time

In [ ]:
df_all_patients = df_all_patients.assign(total_transport_time=lambda x: (x['stablization'] + x['journey'] +x['handover']))

In [ ]:
df_monitor_patients = df_monitor_patients.assign(total_transport_time=lambda x: (x['stablization'] + x['journey'] +x['handover']))

In [ ]:
print('The number of monitoring transports to GOSH is: ',len(df_monitor_patients[df_monitor_patients['dest_is_GOSH'] == 'Yes']))

In [ ]:
df_all_patients.to_csv('df_all_patients.csv')
df_monitor_patients.to_csv('df_monitor_patients.csv')